In [1]:
import AFQ.data as afqd
import AFQ.api as api
import AFQ.mask as afm
import cloudknot as ck
import importlib
import itertools
import os
import s3fs
import bids

/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credenti

In [2]:
ck.set_region('us-west-2')

## Define the preprocessing function

In [3]:
def afq_hbn_subject(subject, reg_subject):
    import AFQ.data as afqd
    import AFQ.mask as afm
    import AFQ.api as api
    import os
    import subprocess
    import sys
    from bids import BIDSLayout
    from s3fs import S3FileSystem

    local_bids_folder = "hbn"
    input_bucket = "fcp-indi"
    input_s3_prefix = "data/Projects/HBN/BIDS_curated"

    # select subjects from qsiprep
    study = afqd.S3BIDSStudy(
        "hbn-curated",
        input_bucket,
        input_s3_prefix,
        subjects=[subject],
        random_seed=42
    )
    study.download(local_bids_folder, include_derivs=True)

    recon_bucket = "hbn-afq"
    recon_s3_prefix = ""

    study = afqd.S3BIDSStudy(
        "hbn-recon",
        recon_bucket,
        recon_s3_prefix,
        subjects=[subject],
        random_seed=42,
        use_participants_tsv=True,
        anon=False,
    )
    study.download(local_bids_folder, include_derivs="qsirecon")

    reg_template = {
        "b0": "mni_t1",
        "power_map": "mni_t2",
        "dti_fa_subject": "dti_fa_template",
        "subject_sls": "hcp_atlas",
    }

    myafq = api.AFQ(
        local_bids_folder,
        dmriprep="qsiprep",
        brain_mask=afm.MaskFile(
            "mask",
            {
                "desc": "brain",
                 "space": None
            }
        ),
        reg_subject=reg_subject,
        reg_template=reg_template[reg_subject],
        scalars=["dki_md", "dki_fa", "dti_md", "dti_fa"],
        custom_tractography_bids_filters={
            "suffix": "ifod2",
            "scope": "qsirecon",
        },
    )

    myafq.export_all()

    output_bucket = "hbn-afq"
    output_s3_prefix = ""

    myafq.upload_to_s3(
        S3FileSystem(),
        os.path.join(
            output_bucket,
            output_s3_prefix,
            "derivatives/afq"
        )
    )

    return {subject: True}

In [4]:
recover_from_config = True

In [5]:
if not recover_from_config:
    di = ck.DockerImage(
        name="afq-hbn-recon",
        func=afq_hbn_subject,
        base_image="libglxvfb:1",
        github_installs="https://github.com/yeatmanlab/pyAFQ.git@master",
        overwrite=True
    )
else:
    di = ck.DockerImage(name="afq-hbn-recon")

## Build, tag, and push the Docker image

In [6]:
if not recover_from_config:
    di.build(tags=["afq-hbn-recon-20210127"])

In [7]:
if not recover_from_config:
    repo = ck.aws.DockerRepo(name=ck.get_ecr_repo())

In [8]:
if not recover_from_config:
    print(repo.repo_uri)

In [9]:
if not recover_from_config:
    # The very first time you run this, this command could take
    # a few hours because the docker image is large
    di.push(repo=repo)

In [10]:
print(di.repo_uri)

454929164628.dkr.ecr.us-west-2.amazonaws.com/cloudknot:afq-hbn-recon-20210127


## Create the Knot

In [11]:
# Specify bid_percentage to use Spot instances
# And make sure the volume size is large enough. 55-60 GB seems about right for HBN preprocessing. YMMV.
# knot = ck.Knot(
#     name=f"hbn-afq-20210127-2",
#     docker_image=di,
#     pars_policies=("AmazonS3FullAccess",),
#     bid_percentage=100,
#     memory=64000,
#     job_def_vcpus=8,
#     max_vcpus=8192,
#     retries=2,
#     volume_size=80,
#     aws_resource_tags={"Project": "HBN-FCP-INDI"},
# )

knot = ck.Knot(name=f"hbn-afq-20210127-2")

## Submit the jobs and check on results

In [12]:
recon_study = afqd.S3BIDSStudy(
    "hbn-recon",
    bucket="hbn-afq",
    s3_prefix="derivatives/qsirecon",
    anon=False,
)

afq_study = afqd.S3BIDSStudy(
    "hbn-afq",
    bucket="hbn-afq",
    s3_prefix="derivatives/afq",
    anon=False,
)

Retrieving subject S3 keys
[########################################] | 100% Completed |  0.3s
Retrieving subject S3 keys
[########################################] | 100% Completed |  0.5s


In [13]:
remaining_subs = list(set(recon_study._all_subjects) - set(afq_study._all_subjects))
print(len(remaining_subs))

43


In [14]:
print(len(afq_study._all_subjects))

2050


In [15]:
reg_subject = ["power_map"]

args = list(itertools.product(
    remaining_subs,
    reg_subject,
))

print(len(args))

43


In [16]:
results = knot.map(args, starmap=True)

In [18]:
ck.set_region('us-west-2')
knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
3b0987f6-7c8f-4666-9357-812267dc3da8        hbn-afq-20210127-2-5        PENDING  
35b27531-0507-4da2-8dd2-106baeb32b3a        hbn-afq-20210127-2-2        FAILED   
d4eff159-e836-4406-8ad9-4c0d0e6696af        hbn-afq-20210127-2-3        FAILED   
1227a559-525e-48c3-9d30-63f098cb7f01        hbn-afq-20210127-2-4        FAILED   
16778ab8-b761-4bfe-8520-4cfe4f73edc2        hbn-afq-20210127-2-0        FAILED   
31e32fc3-fea4-437c-9fe0-2272baa8da35        hbn-afq-20210127-2-1        FAILED   


## Results

The results are dicts where the keys are the subject IDs and the values report sucess or failure

In [ ]:
results.result()

## When you're done, clobber the knot

In [ ]:
knot.clobber(clobber_pars=True)